#### 1. Download the movie dataset from GCS:

In [1]:
%env BASE_DIR=/home/jupyter/prep

env: BASE_DIR=/home/jupyter/prep


In [ ]:
!gsutil cp gs://cs327e-open-access/movies.cypher.zip $BASE_DIR

In [ ]:
!unzip $BASE_DIR/movies.cypher.zip

#### 2. Connect to the database and load the dataset:
##### Note: Change the PW variable before running. 

In [2]:
%env CYPHER=/home/jupyter/neo4j-community-4.3.3/bin/cypher-shell
%env USER=neo4j
%env PW=cs327e!

env: CYPHER=/home/jupyter/neo4j-community-4.3.3/bin/cypher-shell
env: USER=neo4j
env: PW=cs327e!


In [3]:
CONNECT="$CYPHER -u $USER -p $PW"

In [4]:
!{CONNECT} "SHOW DATABASES"

+--------------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default | home  |
+--------------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    | TRUE  |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   | FALSE |
+--------------------------------------------------------------------------------------------------------+

2 rows
ready to start consuming query after 745 ms, results consumed after another 81 ms


In [5]:
!{CONNECT} "MATCH (n) DETACH DELETE n"

0 rows
ready to start consuming query after 178 ms, results consumed after another 0 ms
Deleted 171 nodes, Deleted 253 relationships


In [6]:
!{CONNECT} "MATCH (n) RETURN n"

+---+
| n |
+---+
+---+

0 rows
ready to start consuming query after 22 ms, results consumed after another 1 ms


In [7]:
!cat $BASE_DIR/movies.cypher | $CYPHER -u $USER -p $PW --format plain

a, m, d
(:Person {name: "Tom Hanks", born: 1956}), (:Movie {tagline: "In every life there comes a time when that thing you dream becomes that thing you do", title: "That Thing You Do", released: 1996}), (:Person {name: "Tom Hanks", born: 1956})
(:Person {name: "Tom Hanks", born: 1956}), (:Movie {tagline: "Once in a lifetime you get a chance to do something different.", title: "A League of Their Own", released: 1992}), (:Person {name: "Penny Marshall", born: 1943})
(:Person {name: "Tom Hanks", born: 1956}), (:Movie {tagline: "What if someone you never met, someone you never saw, someone you never knew was the only someone for you?", title: "Sleepless in Seattle", released: 1993}), (:Person {name: "Nora Ephron", born: 1941})
(:Person {name: "Tom Hanks", born: 1956}), (:Movie {tagline: "A stiff drink. A little mascara. A lot of nerve. Who said they couldn't bring down the Soviet empire.", title: "Charlie Wilson's War", released: 2007}), (:Person {name: "Mike Nichols", born: 1931})
(:Perso

#### 3. Explore the data using cypher queries:

In [8]:
!{CONNECT} "MATCH (n) RETURN distinct labels(n), count(n)"

+-----------------------+
| labels(n)  | count(n) |
+-----------------------+
| ["Movie"]  | 38       |
| ["Person"] | 133      |
+-----------------------+

2 rows
ready to start consuming query after 78 ms, results consumed after another 37 ms


In [16]:
!{CONNECT} "MATCH (n) RETURN n LIMIT 10"

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| n                                                                                                                                                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Joel Silver", born: 1952})                                                                                                                                                                                                  |
| (:Person {name: "Emil Eifrem", bor

In [17]:
!{CONNECT} "MATCH ()-[r]->() RETURN distinct type(r) LIMIT 10"

+------------+
| type(r)    |
+------------+
| "PRODUCED" |
| "ACTED_IN" |
| "DIRECTED" |
| "WROTE"    |
| "FOLLOWS"  |
| "REVIEWED" |
+------------+

6 rows
ready to start consuming query after 47 ms, results consumed after another 21 ms


In [18]:
!{CONNECT} "MATCH (m:Movie) RETURN m.title, m.released ORDER BY m.released LIMIT 10"

+------------------------------------------------+
| m.title                           | m.released |
+------------------------------------------------+
| "One Flew Over the Cuckoo's Nest" | 1975       |
| "Top Gun"                         | 1986       |
| "Stand By Me"                     | 1986       |
| "Joe Versus the Volcano"          | 1990       |
| "A League of Their Own"           | 1992       |
| "Hoffa"                           | 1992       |
| "A Few Good Men"                  | 1992       |
| "Unforgiven"                      | 1992       |
| "Sleepless in Seattle"            | 1993       |
| "Johnny Mnemonic"                 | 1995       |
+------------------------------------------------+

10 rows
ready to start consuming query after 46 ms, results consumed after another 3 ms


In [20]:
!{CONNECT} "MATCH (m:Movie) WHERE m.released >= 1995 AND m.released <= 2000 RETURN m.title, m.released ORDER BY m.released"

+---------------------------------------+
| m.title                  | m.released |
+---------------------------------------+
| "Johnny Mnemonic"        | 1995       |
| "Apollo 13"              | 1995       |
| "That Thing You Do"      | 1996       |
| "The Birdcage"           | 1996       |
| "Twister"                | 1996       |
| "The Devil's Advocate"   | 1997       |
| "As Good as It Gets"     | 1997       |
| "What Dreams May Come"   | 1998       |
| "You've Got Mail"        | 1998       |
| "When Harry Met Sally"   | 1998       |
| "The Matrix"             | 1999       |
| "Snow Falling on Cedars" | 1999       |
| "The Green Mile"         | 1999       |
| "Bicentennial Man"       | 1999       |
| "Jerry Maguire"          | 2000       |
| "The Replacements"       | 2000       |
| "Cast Away"              | 2000       |
+---------------------------------------+

17 rows
ready to start consuming query after 21 ms, results consumed after another 4 ms


#### Note the double curly braces {}

In [21]:
!{CONNECT} "MATCH (p:Person)-[r]->(m:Movie {{title: 'Jerry Maguire'}}) RETURN p, r LIMIT 10;"

+--------------------------------------------------------------------------------------------------------------+
| p                                                 | r                                                        |
+--------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Jerry O'Connell", born: 1974})   | [:ACTED_IN {roles: ["Frank Cushman"]}]                   |
| (:Person {name: "Cameron Crowe", born: 1957})     | [:PRODUCED]                                              |
| (:Person {name: "Bonnie Hunt", born: 1961})       | [:ACTED_IN {roles: ["Laurel Boyd"]}]                     |
| (:Person {name: "Cameron Crowe", born: 1957})     | [:DIRECTED]                                              |
| (:Person {name: "Jay Mohr", born: 1970})          | [:ACTED_IN {roles: ["Bob Sugar"]}]                       |
| (:Person {name: "Cameron Crowe", born: 1957})     | [:WROTE]                                  

In [22]:
!{CONNECT} "MATCH (t:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m) RETURN m.title, m.released ORDER BY m.released"

+---------------------------------------+
| m.title                  | m.released |
+---------------------------------------+
| "Joe Versus the Volcano" | 1990       |
| "A League of Their Own"  | 1992       |
| "Sleepless in Seattle"   | 1993       |
| "Apollo 13"              | 1995       |
| "That Thing You Do"      | 1996       |
| "You've Got Mail"        | 1998       |
| "The Green Mile"         | 1999       |
| "Cast Away"              | 2000       |
| "The Polar Express"      | 2004       |
| "The Da Vinci Code"      | 2006       |
| "Charlie Wilson's War"   | 2007       |
| "Cloud Atlas"            | 2012       |
+---------------------------------------+

12 rows
ready to start consuming query after 47 ms, results consumed after another 3 ms


#### Prompt 1: Which person directed the movie Apollo 13? Return the director's name.

In [23]:
!{CONNECT} "MATCH (m {{title: 'Apollo 13'}})<-[:DIRECTED]-(d) RETURN d.name"

+--------------+
| d.name       |
+--------------+
| "Ron Howard" |
+--------------+

1 row
ready to start consuming query after 26 ms, results consumed after another 4 ms


In [6]:
!{CONNECT} "MATCH (m)<-[:DIRECTED]-(d) where m.title = 'Apollo 13' RETURN d.name"

+--------------+
| d.name       |
+--------------+
| "Ron Howard" |
+--------------+

1 row
ready to start consuming query after 23 ms, results consumed after another 1 ms


#### Prompt 2: Which persons worked on the movie Jerry Maguire? Return their name and role in the movie. Limit result count to 10. 

In [26]:
!{CONNECT} "MATCH (p:Person)-[r]->(:Movie {{title: 'Jerry Maguire'}}) RETURN p.name, type(r), r.roles LIMIT 10"

+------------------------------------------------------+
| p.name              | type(r)    | r.roles           |
+------------------------------------------------------+
| "Jerry O'Connell"   | "ACTED_IN" | ["Frank Cushman"] |
| "Cameron Crowe"     | "PRODUCED" | NULL              |
| "Bonnie Hunt"       | "ACTED_IN" | ["Laurel Boyd"]   |
| "Cameron Crowe"     | "DIRECTED" | NULL              |
| "Jay Mohr"          | "ACTED_IN" | ["Bob Sugar"]     |
| "Cameron Crowe"     | "WROTE"    | NULL              |
| "Cuba Gooding Jr."  | "ACTED_IN" | ["Rod Tidwell"]   |
| "Jessica Thompson"  | "REVIEWED" | NULL              |
| "Jonathan Lipnicki" | "ACTED_IN" | ["Ray Boyd"]      |
| "Renee Zellweger"   | "ACTED_IN" | ["Dorothy Boyd"]  |
+------------------------------------------------------+

10 rows
ready to start consuming query after 39 ms, results consumed after another 2 ms


#### Prompt 3: Which persons acted in the movie Jerry Maguire? Return their name and role in the movie. 
#### Hint: use the relationship type: `ACTED_IN`.

In [27]:
!{CONNECT} "MATCH (p:Person)-[r:ACTED_IN]->(:Movie {{title: 'Jerry Maguire'}}) RETURN p.name, type(r), r.roles"

+-------------------------------------------------------+
| p.name              | type(r)    | r.roles            |
+-------------------------------------------------------+
| "Jerry O'Connell"   | "ACTED_IN" | ["Frank Cushman"]  |
| "Bonnie Hunt"       | "ACTED_IN" | ["Laurel Boyd"]    |
| "Jay Mohr"          | "ACTED_IN" | ["Bob Sugar"]      |
| "Cuba Gooding Jr."  | "ACTED_IN" | ["Rod Tidwell"]    |
| "Jonathan Lipnicki" | "ACTED_IN" | ["Ray Boyd"]       |
| "Renee Zellweger"   | "ACTED_IN" | ["Dorothy Boyd"]   |
| "Kelly Preston"     | "ACTED_IN" | ["Avery Bishop"]   |
| "Regina King"       | "ACTED_IN" | ["Marcee Tidwell"] |
| "Tom Cruise"        | "ACTED_IN" | ["Jerry Maguire"]  |
+-------------------------------------------------------+

9 rows
ready to start consuming query after 39 ms, results consumed after another 2 ms


#### Prompt 4: Which persons acted in a movie with Kevin Bacon? Return the movie's title, Kevin Bacon's role and the other person's role in the same movie. Limit result count to 10. 

In [12]:
!{CONNECT} "MATCH (p1:Person {{name: 'Kevin Bacon'}})-[r:ACTED_IN]->(m:Movie)<-[r2:ACTED_IN]-(p2:Person) RETURN m.title, r.roles as kb_role, p2.name, r2.roles LIMIT 10"

+------------------------------------------------------------------------------------+
| m.title          | kb_role             | p2.name          | r2.roles               |
+------------------------------------------------------------------------------------+
| "Apollo 13"      | ["Jack Swigert"]    | "Tom Hanks"      | ["Jim Lovell"]         |
| "Apollo 13"      | ["Jack Swigert"]    | "Ed Harris"      | ["Gene Kranz"]         |
| "Apollo 13"      | ["Jack Swigert"]    | "Gary Sinise"    | ["Ken Mattingly"]      |
| "Apollo 13"      | ["Jack Swigert"]    | "Bill Paxton"    | ["Fred Haise"]         |
| "Frost/Nixon"    | ["Jack Brennan"]    | "Sam Rockwell"   | ["James Reston, Jr."]  |
| "Frost/Nixon"    | ["Jack Brennan"]    | "Michael Sheen"  | ["David Frost"]        |
| "Frost/Nixon"    | ["Jack Brennan"]    | "Frank Langella" | ["Richard Nixon"]      |
| "Frost/Nixon"    | ["Jack Brennan"]    | "Oliver Platt"   | ["Bob Zelnick"]        |
| "A Few Good Men" | ["Capt. Jack Ross"] | 

#### Unfortunately, the Community Edition doesn't let us create multiple databases. You'll need to delete the movie graph before creating a different graph in our database. To delete the graph, use this command:

`!{CONNECT} "MATCH (n) DETACH DELETE n"`

In [29]:
!{CONNECT} "MATCH (n) DETACH DELETE n"

0 rows
ready to start consuming query after 43 ms, results consumed after another 0 ms
Deleted 171 nodes, Deleted 253 relationships


In [30]:
!{CONNECT} "MATCH (n) RETURN n"

+---+
| n |
+---+
+---+

0 rows
ready to start consuming query after 2 ms, results consumed after another 1 ms
